<a href="https://colab.research.google.com/github/Jazzystic/Data-Source-API-Analyst-Test/blob/main/migraci%C3%B3n_hist%C3%B3rico_rankingstable_GFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Subir archivo
print("Selecciona rankings_table.csv:")
uploaded = files.upload()

Selecciona rankings_table.csv:


Saving rankings_table.csv to rankings_table.csv


In [2]:
import pandas as pd

# Leer el CSV
df = pd.read_csv('rankings_table.csv',
                 encoding='utf-8',
                 on_bad_lines='warn',
                 engine='python')

print(f"Total de filas: {len(df)}")
print(f"\nColumnas detectadas ({len(df.columns)}):")
print(df.columns.tolist())
print("\nPrimeras 3 filas:")
print(df.head(3))
print("\nValores nulos por columna:")
print(df.isnull().sum())
print("\nTipos de datos:")
print(df.dtypes)

Total de filas: 19422

Columnas detectadas (9):
['Digital Audit CS Column Name', 'Market #', 'AuditFor', 'Source', 'Period', 'Attribute', 'Ranking Value', 'Rating Value', 'Active Selection']

Primeras 3 filas:
                Digital Audit CS Column Name  Market #  \
0  Casa Dorada Los Cabos, Resort & Spa (141)       NaN   
1                              ME Cabo (162)       NaN   
2                                          0       NaN   

                              AuditFor       Source  Period  \
0  Casa Dorada Los Cabos, Resort & Spa  TripAdvisor  Feb-21   
1  Casa Dorada Los Cabos, Resort & Spa  TripAdvisor  Feb-21   
2  Casa Dorada Los Cabos, Resort & Spa  TripAdvisor  Feb-21   

                                  Attribute  Ranking Value  Rating Value  \
0        Casa Dorada Los Cabos Resort & Spa           38.0           NaN   
1                                   ME Cabo           22.0           NaN   
2  JW Marriott Los Cabos Beach Resort & Spa            0.0           NaN   


In [4]:
import psycopg2
from psycopg2.extras import execute_batch
import pandas as pd

# df ya tiene los datos cargados del CSV

# Limpiar datos
df_clean = df.copy()

# Renombrar columnas para que coincidan con PostgreSQL
column_mapping = {
    'Digital Audit CS Column Name': 'digital_audit_cs_column_name',
    'Market #': 'market_number',
    'AuditFor': 'audit_for',
    'Source': 'source',
    'Period': 'period',
    'Attribute': 'attribute',
    'Ranking Value': 'ranking_value',
    'Rating Value': 'rating_value',
    'Active Selection': 'active_selection'
}

df_clean = df_clean.rename(columns=column_mapping)

# Convertir tipos de datos
df_clean['market_number'] = pd.to_numeric(df_clean['market_number'], errors='coerce').astype('Int64')
df_clean['ranking_value'] = pd.to_numeric(df_clean['ranking_value'], errors='coerce')
df_clean['rating_value'] = pd.to_numeric(df_clean['rating_value'], errors='coerce')

# Verificar duplicados
duplicados = df_clean.duplicated(subset=['audit_for', 'source', 'period', 'attribute'], keep=False)
print(f"Filas duplicadas: {duplicados.sum()}")

if duplicados.sum() > 0:
    print("\nEliminando duplicados...")
    df_clean = df_clean.drop_duplicates(subset=['audit_for', 'source', 'period', 'attribute'], keep='first')
    print(f"Filas después de limpiar: {len(df_clean)}")

print(f"\nDatos preparados: {len(df_clean)} filas")

# Preparar datos para inserción
columns_order = ['digital_audit_cs_column_name', 'market_number', 'audit_for', 'source',
                 'period', 'attribute', 'ranking_value', 'rating_value', 'active_selection']

data = []
for _, row in df_clean[columns_order].iterrows():
    tuple_data = []
    for val in row:
        if pd.isna(val) or val is pd.NA:
            tuple_data.append(None)
        else:
            tuple_data.append(val)
    data.append(tuple(tuple_data))

# Conectar a PostgreSQL
conn = psycopg2.connect(
    host="gfg-postgresql-server-2025.postgres.database.azure.com",
    database="gfg_asset_management",
    user="svc_n8n_admin",
    password="PowerBI2025!",
    sslmode="require"
)

# SQL de inserción
insert_sql = """
INSERT INTO raw_digital_presence.rankings_table
(digital_audit_cs_column_name, market_number, audit_for, source, period,
 attribute, ranking_value, rating_value, active_selection)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Insertar en batch
cursor = conn.cursor()
print("\nInsertando datos en PostgreSQL...")
execute_batch(cursor, insert_sql, data, page_size=1000)
conn.commit()

print(f"\n{len(data)} filas insertadas exitosamente en rankings_table")

cursor.close()
conn.close()

print("\nMigracion completada!")

Filas duplicadas: 0

Datos preparados: 19422 filas

Insertando datos en PostgreSQL...

19422 filas insertadas exitosamente en rankings_table

Migracion completada!
